In [176]:
#libraries
import numpy as np 
import pandas as pd

**Вариант 2. Задание 1.**

\begin{equation*}
 \begin{cases}
    \frac{\partial u}{\partial t} = \frac{\partial}{\partial x}(u\frac{\partial u}{\partial x}) + \frac{\partial}{\partial y}(u\frac{\partial u}{\partial y}) , 0 < t\le 1, 0 < x,y < 1 \\
    u(0, x, y) = \frac{(1 + x + y)^2}{13}, 0 \le x,y \le 1 \\
    u(t. 0, y) = \frac{(1 + y)^2}{(13 - 12t)}, 0 < t \le 1, 0 \le y \le 1\\
    u(t, 1, y) = \frac{(2 + y)^2}{(13 - 12t)}, 0 < t \le 1, 0 \le y \le 1\\
    u(t, x, 0) = \frac{(1 + x)^2}{(13 - 12t)}, 0 < t \le 1, 0 < x \le 1\\
    u(t, x, 1) = \frac{(2 + x)^2}{(13 - 12t)}, 0 < t \le 1, 0 < x \le 1\\   
 \end{cases}
\end{equation*}

In [177]:
def f_private(t, r, phi):
    '''
    функция частного решения
    '''
    return (1 + r + phi)**2 / (13 - 12 * t)

def private_solution(t, r, phi):
    '''
    частное решение на сетке
    '''
    u = np.zeros((len(phi), len(r)))
    for i in range(len(phi)):
        u[i, :] = f_private(t, r, phi[i])
    return u

In [178]:
def grid(L, M, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, 1, M), np.linspace(0, 1, N)#менять первый и второй параметр, если изменятся границы

def sweep_mthd(a, b, c, d, m, l, u_ex):
    '''
    Метод прогонки
    '''
    alpha = np.zeros((m, l))
    beta = np.zeros((m, l))
    beta[1:-1, 0] = u_ex[1:-1,0]
    x = u_ex.copy()
    
    for i in range(1, l - 1):
        alpha[1:-1,i] = (-a(i) / (c(i) * alpha[1:-1,i-1] + b(i)))
        beta[1:-1,i] = (d(i) - c(i) * beta[1:-1,i-1]) / (b(i) + c(i) * alpha[1:-1,i-1])
    
    for i in reversed(range(1, l-1)):
        x[1:-1,i] = alpha[1:-1,i] * x[1:-1,i + 1]  + beta[1:-1,i]

    return x

def accuracy(u1, u2):
    '''
    Вычислят точность
    Returns bool
    '''
    max = 0
    epsilon = 0.01#точность
    for m in range(u1.shape[0]):
        for l in range(u1.shape[1]):
            if u2[m][l] != 0:
                num = abs((u2[m][l] - u1[m][l]) / u2[m][l])
                if num > max:
                    max = num
            if max > epsilon:
                return False
    return True

def first_step(u, u1, r, m, n, mu, tau, hr):
    
    def a_l(l):
        return -(u1[1:-1, l + 1]**mu + u1[1:-1, l]**mu) * tau / (2 * hr**2)
    def c_l(l):
        return -(u1[1:-1, l]**mu + u1[1:-1, l - 1]**mu) * tau / (2 * hr**2)
    def b_l(l):    
        return 1 - a_l(l) - c_l(l)
    def d_l(l):
        return u[n, 1:- 1, l]
    
    return sweep_mthd(a_l, b_l, c_l, d_l, m, len(r), u[n+1,:,:])

def second_step(u, u_tilda, u_f, r, m, mu, tau, hf):

    u1 = u.T
    u_tilda1 = u_tilda.T

    def a_m(m):
        return -(u1[1:-1, m + 1]**mu + u1[1:-1, m]**mu) * tau / (2 * hf**2)
    def c_m(m):
        return -(u1[1:-1, m]**mu + u1[1:-1, m - 1]**mu) * tau / (2 * hf**2)
    def b_m(m):
        return 1 - a_m(m) - c_m(m)
    def d_m(m):
        return u_tilda1[1:-1, m]
    
    return sweep_mthd(a_m, b_m, c_m, d_m, len(r), m, u_f.T).T

    
    
def numerical_solve(r, phi, t, mu):
    hr = 1 / (len(r) - 1)
    hf = 1 / (len(phi) - 1)
    tau = 1 / (len(t) - 1)
    
    m = len(phi)
    
    u = np.zeros((len(t), len(phi), len(r)))
    
    for i in range(len(phi)):#начальное условие
        u[0, i, :] = f_private(0, r, phi[i])
    
    for i in range(1, len(t)):
        u[i, :, 0] = f_private(t[i], r[0], phi)#первое граничное условие
        u[i, :, -1] = f_private(t[i], r[-1], phi)#второе граничное условие
        u[i, 0, 1:-1] = f_private(t[i], r[1:-1], phi[0])#третье граничное условие
        u[i, -1, 1:-1] = f_private(t[i], r[1:-1], phi[-1])#третье граничное условие
   
    for i in range(len(t) - 1):
        u_smth = u[i].copy()
        
        while True:
            u_tilda = first_step(u, u_smth, r, m, i, mu, tau, hr)
            u_end = second_step(u_smth, u_tilda, u[i+1], r, m, mu, tau, hf)
            if accuracy(u_smth, u_end):
                u[i + 1] = u_end
                break
            else:
                u_smth = u_end
             
    return u

In [179]:
#ТО ШО МЕНЯТЬ
L = M = 11
N = 71
mu = 1
T = 1

#находим аналитическое решение
r_rep, phi_rep, _ = grid(L, M, N) 

u_private = private_solution(T, r_rep, phi_rep)


#находим численное рашение
r, phi, t = grid(L, M, N)
u_numeric = numerical_solve(r, phi, t, mu)

    
print('Max error:', np.max(np.abs(u_private - u_numeric[-1])))



Max error: 0.29948863544906335


In [180]:
print('_____Analytical____')
f = pd.DataFrame(u_private, columns=r_rep, index=phi_rep)
f

_____Analytical____


,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.0,1.00,1.21,1.44,1.69,1.96,2.25,2.56,2.89,3.24,3.61,4.00
0.1,1.21,1.44,1.69,1.96,2.25,2.56,2.89,3.24,3.61,4.00,4.41
0.2,1.44,1.69,1.96,2.25,2.56,2.89,3.24,3.61,4.00,4.41,4.84
0.3,1.69,1.96,2.25,2.56,2.89,3.24,3.61,4.00,4.41,4.84,5.29
0.4,1.96,2.25,2.56,2.89,3.24,3.61,4.00,4.41,4.84,5.29,5.76
0.5,2.25,2.56,2.89,3.24,3.61,4.00,4.41,4.84,5.29,5.76,6.25
0.6,2.56,2.89,3.24,3.61,4.00,4.41,4.84,5.29,5.76,6.25,6.76
0.7,2.89,3.24,3.61,4.00,4.41,4.84,5.29,5.76,6.25,6.76,7.29
0.8,3.24,3.61,4.00,4.41,4.84,5.29,5.76,6.25,6.76,7.29,7.84
0.9,3.61,4.00,4.41,4.84,5.29,5.76,6.25,6.76,7.29,7.84,8.41


In [181]:
print('_____Numerical____')
f1 = pd.DataFrame(u_numeric[-1], columns=r_rep, index=phi_rep)
f1


_____Numerical____


,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.0,1.00,1.210000,1.440000,1.690000,1.960000,2.250000,2.560000,2.890000,3.240000,3.610000,4.00
0.1,1.21,1.520366,1.771459,2.044824,2.339853,2.656305,2.994121,3.353349,3.734108,4.136564,4.41
0.2,1.44,1.815912,2.089708,2.386219,2.704982,3.045782,3.408570,3.793408,4.200442,4.629880,4.84
0.3,1.69,2.113457,2.408409,2.726578,3.067490,3.430895,3.816707,4.224968,4.655816,5.109465,5.29
0.4,1.96,2.419108,2.733369,3.071673,3.433320,3.817916,4.225292,4.655444,5.108489,5.584630,5.76
0.5,2.25,2.734705,3.066853,3.424069,3.805278,4.209882,4.637599,5.088363,5.562261,6.059489,6.25
0.6,2.56,3.059972,3.409249,3.784545,4.184388,4.607960,5.054865,5.524975,6.018347,6.535164,6.76
0.7,2.89,3.393302,3.759769,4.152776,4.570555,5.012134,5.477032,5.965078,6.476298,7.010857,7.29
0.8,3.24,3.732048,4.116772,4.527661,4.962921,5.421576,5.903142,6.407435,6.934461,7.484354,7.84
0.9,3.61,4.072544,4.477881,4.907487,5.360044,5.834847,6.331544,6.849998,7.390206,7.952252,8.41


In [182]:
print('_____Errors____')
fe =  pd.DataFrame(np.abs(u_private - u_numeric[-1]), columns=r_rep, index=phi_rep)
fe

_____Errors____


,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
0.1,0.0,0.080366,0.081459,0.084824,0.089853,0.096305,0.104121,0.113349,0.124108,0.136564,0.0
0.2,0.0,0.125912,0.129708,0.136219,0.144982,0.155782,0.168570,0.183408,0.200442,0.219880,0.0
0.3,0.0,0.153457,0.158409,0.166578,0.177490,0.190895,0.206707,0.224968,0.245816,0.269465,0.0
0.4,0.0,0.169108,0.173369,0.181673,0.193320,0.207916,0.225292,0.245444,0.268489,0.294630,0.0
0.5,0.0,0.174705,0.176853,0.184069,0.195278,0.209882,0.227599,0.248363,0.272261,0.299489,0.0
0.6,0.0,0.169972,0.169249,0.174545,0.184388,0.197960,0.214865,0.234975,0.258347,0.285164,0.0
0.7,0.0,0.153302,0.149769,0.152776,0.160555,0.172134,0.187032,0.205078,0.226298,0.250857,0.0
0.8,0.0,0.122048,0.116772,0.117661,0.122921,0.131576,0.143142,0.157435,0.174461,0.194354,0.0
0.9,0.0,0.072544,0.067881,0.067487,0.070044,0.074847,0.081544,0.089998,0.100206,0.112252,0.0
